# Import Julia Packages for Benchmarking
---

In [1]:
using BenchmarkTools
using StateSpaceDynamics
using Random
using Statistics
using PythonCall
using DataFrames
using Printf
using LinearAlgebra
using CSV

# Import Python Packages for Benchmarking
---

In [2]:
# Import Python packages
const dynamax = pyimport("dynamax.linear_gaussian_ssm")
const jr = pyimport("jax.random")
const pykalman = pyimport("pykalman")
const np = pyimport("numpy")

Python: <module 'numpy' from 'c:\\Users\\ryansenne\\Documents\\GitHub\\ssm_julia\\benchmarking\\.CondaPkg\\env\\Lib\\site-packages\\numpy\\__init__.py'>

# Benchmarking Functions
---

In [3]:

struct BenchConfig
    latent_dims::Vector{Int}
    obs_dims::Vector{Int} 
    seq_lengths::Vector{Int}
    n_iters::Int
    n_repeats::Int
end

default_config = BenchConfig(
    [2, 4, 8, 16],       # latent dimensions
    [4, 8, 16, 32],      # observation dimensions
    [100, 500, 1000],    # sequence lengths
    100,                 # EM iterations
    5                    # benchmark repeats
)

"""
Generate random parameters and data for benchmarking
"""
function generate_test_data(latent_dim::Int, obs_dim::Int, seq_len::Int)
    # Generate random model parameters
    A = randn(latent_dim, latent_dim)
    C = randn(obs_dim, latent_dim)
    Q = Matrix(Diagonal(ones(latent_dim)))
    R = Matrix(Diagonal(ones(obs_dim)))
    x0 = zeros(latent_dim)
    P0 = Matrix(Diagonal(ones(latent_dim)))
    
    # Create models
    lds_julia = GaussianLDS(
        A=A, C=C, Q=Q, R=R, x0=x0, P0=P0,
        obs_dim=obs_dim, latent_dim=latent_dim
    )
    
    # Generate data
    x, y = sample(lds_julia, seq_len, 1)
    
    return lds_julia, x, y, (A, C, Q, R, x0, P0)
end

"""
Run a single benchmark trial with error handling
"""
function run_single_benchmark(model_type::Symbol, lds_julia, y, y_np, params=nothing; config=default_config)
    try
        if model_type == :julia
            bench = @benchmark fit!($lds_julia, $y, max_iter=$config.n_iters) samples=config.n_repeats
            return (time=minimum(bench).time, memory=bench.memory, allocs=bench.allocs, success=true)
            
        elseif model_type == :dynamax
            A_np, C_np, Q_np, R_np, x0_np = params
            
            # Get dimensions directly from numpy arrays
            latent_dim = pyconvert(Int, A_np.shape[0])
            obs_dim = pyconvert(Int, C_np.shape[0])
            
            lds_dynamax = dynamax.LinearGaussianSSM(latent_dim, obs_dim)
            key = jr.PRNGKey(0)
            test_params, param_props = lds_dynamax.initialize(
                key, dynamics_weights=A_np, dynamics_covariance=Q_np,
                emission_weights=C_np, emission_covariance=R_np,
                initial_mean=x0_np
            )
            bench = @benchmark $lds_dynamax.fit_em(
                $test_params, $param_props, $y_np, 
                num_iters=$config.n_iters
            ) samples=config.n_repeats
            return (time=minimum(bench).time, memory=0, allocs=0, success=true)
            
        elseif model_type == :pykalman
            A_np, C_np, Q_np, R_np, x0_np = params
            
            # Get dimensions directly from numpy arrays
            latent_dim = pyconvert(Int, A_np.shape[0])
            obs_dim = pyconvert(Int, C_np.shape[0])
            
            kf = pykalman.KalmanFilter(
                n_dim_state=latent_dim, n_dim_obs=obs_dim,
                transition_matrices=A_np, observation_matrices=C_np,
                transition_covariance=Q_np, observation_covariance=R_np,
                initial_state_mean=x0_np
            )
            bench = @benchmark $kf.em($y_np, n_iter=$config.n_iters) samples=config.n_repeats
            return (time=minimum(bench).time, memory=0, allocs=0, success=true)
        end
    catch e
        @warn "Benchmark failed for $model_type" exception=e stacktrace=catch_backtrace()
        return (time=NaN, memory=0, allocs=0, success=false)
    end
end

"""
Benchmark model fitting with separate runs for each package
"""
function benchmark_fitting(config::BenchConfig=default_config)
    results = []
    
    for latent_dim in config.latent_dims
        for obs_dim in config.obs_dims
            obs_dim < latent_dim && continue
            
            for seq_len in config.seq_lengths
                println("\nTesting configuration: latent_dim=$latent_dim, obs_dim=$obs_dim, seq_len=$seq_len")
                
                # Generate test data once
                lds_julia, x, y, params = generate_test_data(latent_dim, obs_dim, seq_len)
                A, C, Q, R, x0, _ = params

                # Convert all arrays to numpy
                A_np = np.array(A)
                C_np = np.array(C)
                Q_np = np.array(Q)
                R_np = np.array(R)
                x0_np = np.array(x0)
                y_np = np.array(permutedims(dropdims(y, dims=3), (2, 1)))
                
                numpy_params = (A_np, C_np, Q_np, R_np, x0_np)

                # Run benchmarks separately
                julia_result = run_single_benchmark(:julia, lds_julia, y, y_np; config=config)
                dynamax_result = run_single_benchmark(:dynamax, lds_julia, y, y_np, numpy_params; config=config)
                pykalman_result = run_single_benchmark(:pykalman, lds_julia, y, y_np, numpy_params; config=config)
                
                push!(results, Dict(
                    "config" => (latent_dim=latent_dim, obs_dim=obs_dim, seq_len=seq_len),
                    "julia" => julia_result,
                    "dynamax" => dynamax_result,
                    "pykalman" => pykalman_result
                ))
                
                # Print immediate results for this configuration
                print_single_result(results[end])
            end
        end
    end
    
    return results
end

"""
Print results for a single configuration
"""
function print_single_result(result)
    config = result["config"]
    println("\nConfiguration:")
    println("  Latent dim: $(config.latent_dim)")
    println("  Obs dim: $(config.obs_dim)")
    println("  Seq length: $(config.seq_len)")
    
    for pkg in [:julia, :dynamax, :pykalman]
        r = result[string(pkg)]
        if r.success
            @printf("\n%s timing: %.3f seconds", uppercase(string(pkg)), r.time/1e9)
            if pkg == :julia
                println("\n  Memory: $(r.memory) bytes")
                println("  Allocations: $(r.allocs)")
            end
        else
            println("\n$(uppercase(string(pkg))) benchmark failed")
        end
    end
    println("\n" * "-"^50)
end

print_single_result

In [ ]:
results = benchmark_fitting()


Testing configuration: latent_dim=2, obs_dim=4, seq_len=100


Fitting LDS via EM...   2%|██                                                |  ETA: 0:09:01 ( 5.52  s/it)┌ Warning: Benchmark failed for julia
│   exception = CompositeException(Any[TaskFailedException(Task (failed) @0x0000018782e8ba30)])
│   stacktrace = Union{Ptr{Nothing}, Base.InterpreterIP}[Ptr{Nothing} @0x0000018727efac1b, Ptr{Nothing} @0x0000018711b8a74a, Ptr{Nothing} @0x0000018711b9ae5a, Ptr{Nothing} @0x0000018711b9b332, Ptr{Nothing} @0x0000018711b9b4f9, Ptr{Nothing} @0x0000018711b9b80c, Ptr{Nothing} @0x0000018727ed7e7f, Ptr{Nothing} @0x0000018727ed8159, Ptr{Nothing} @0x0000018727ed818f, Ptr{Nothing} @0x00007ffd621f708a, Ptr{Nothing} @0x0000018727ec4f5d, Ptr{Nothing} @0x0000018727ed0521, Ptr{Nothing} @0x0000018727ed1611, Ptr{Nothing} @0x0000018727ed1c85, Ptr{Nothing} @0x0000018727ed1cc4, Ptr{Nothing} @0x00007ffd62206c3a, Ptr{Nothing} @0x00007ffd6220675c, Ptr{Nothing} @0x00007ffd622072f8, Ptr{Nothing} @0x00007ffd622080fe, Base.InterpreterIP in top-level CodeInfo for Main at stat

 |----------------------------------------| 0.00% [0/100 00:00<?] |----------------------------------------| 1.00% [1/100 00:00<01:25] |----------------------------------------| 2.00% [2/100 00:00<00:42] |X---------------------------------------| 3.00% [3/100 00:00<00:28] |X---------------------------------------| 4.00% [4/100 00:00<00:20] |XX--------------------------------------| 5.00% [5/100 00:00<00:16] |XX--------------------------------------| 6.00% [6/100 00:00<00:13] |XX--------------------------------------| 7.00% [7/100 00:00<00:11] |XXX-------------------------------------| 8.00% [8/100 00:00<00:10] |XXX-------------------------------------| 9.00% [9/100 00:00<00:08] |XXXX------------------------------------| 11.00% [11/100 00:00<00:07] |XXXXX-----------------------------------| 13.00% [13/100 00:00<00:05] |XXXXXX----------------------------------| 15.00% [15/100 00:00<00:04] |XXXXXXX---------------------------------| 18.00% [18/100 00:00<00:03] |XXXXXXXX--------------------

Fitting LDS via EM... 100%|██████████████████████████████████████████████████| Time: 0:00:01 (17.04 ms/it)
Fitting LDS via EM... 100%|██████████████████████████████████████████████████| Time: 0:00:01 (15.05 ms/it)
Fitting LDS via EM... 100%|██████████████████████████████████████████████████| Time: 0:00:01 (12.64 ms/it)
Fitting LDS via EM... 100%|██████████████████████████████████████████████████| Time: 0:00:01 (12.84 ms/it)
Fitting LDS via EM... 100%|██████████████████████████████████████████████████| Time: 0:00:01 (12.76 ms/it)
Fitting LDS via EM... 100%|██████████████████████████████████████████████████| Time: 0:00:01 (12.46 ms/it)
Fitting LDS via EM... 100%|██████████████████████████████████████████████████| Time: 0:00:01 (12.09 ms/it)
Fitting LDS via EM... 100%|██████████████████████████████████████████████████| Time: 0:00:01 (12.95 ms/it)
Fitting LDS via EM... 100%|██████████████████████████████████████████████████| Time: 0:00:01 (12.86 ms/it)
Fitting LDS via EM... 100%|██████████

 |----------------------------------------| 0.00% [0/100 00:00<?] |----------------------------------------| 1.00% [1/100 00:00<00:34] |----------------------------------------| 2.00% [2/100 00:00<00:17] |X---------------------------------------| 3.00% [3/100 00:00<00:11] |X---------------------------------------| 4.00% [4/100 00:00<00:08] |XX--------------------------------------| 5.00% [5/100 00:00<00:06] |XX--------------------------------------| 7.00% [7/100 00:00<00:04] |XXXX------------------------------------| 10.00% [10/100 00:00<00:03] |XXXXXX----------------------------------| 15.00% [15/100 00:00<00:01] |XXXXXXXXX-------------------------------| 23.00% [23/100 00:00<00:01] |XXXXXXXXXXXXXX--------------------------| 36.00% [36/100 00:00<00:00] |XXXXXXXXXXXXXXXXXXXXXX------------------| 55.00% [55/100 00:00<00:00] |XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX---------| 79.00% [79/100 00:00<00:00] |XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX| 100.00% [100/100 00:00<00:00] |--------------------

...

┌ Warning: Benchmark failed for julia
│   exception = CompositeException(Any[TaskFailedException(Task (failed) @0x0000018782cc4da0)])
│   stacktrace = Union{Ptr{Nothing}, Base.InterpreterIP}[Ptr{Nothing} @0x0000018727efac1b, Ptr{Nothing} @0x0000018711b8a74a, Ptr{Nothing} @0x0000018711b9ae5a, Ptr{Nothing} @0x0000018711c2f772, Ptr{Nothing} @0x0000018711c2f939, Ptr{Nothing} @0x0000018711c2fc4c, Ptr{Nothing} @0x0000018727ed7e7f, Ptr{Nothing} @0x0000018727ed8159, Ptr{Nothing} @0x0000018727ed818f, Ptr{Nothing} @0x00007ffd621f708a, Ptr{Nothing} @0x0000018727ec4f5d, Ptr{Nothing} @0x0000018727ed0521, Ptr{Nothing} @0x0000018727ed1611, Ptr{Nothing} @0x0000018727ed1c85, Ptr{Nothing} @0x0000018727ed1cc4, Ptr{Nothing} @0x00007ffd62206c3a, Ptr{Nothing} @0x00007ffd6220675c, Ptr{Nothing} @0x00007ffd622072f8, Ptr{Nothing} @0x00007ffd622080fe, Base.InterpreterIP in top-level CodeInfo for Main at statement 0, Ptr{Nothing} @0x00007ffd62223ccb, Ptr{Nothing} @0x00007ffd6222463e, Ptr{Nothing} @0x00007ffd62225

 |----------------------------------------| 0.00% [0/100 00:00<?] |----------------------------------------| 1.00% [1/100 00:00<00:52] |----------------------------------------| 2.00% [2/100 00:00<00:26] |X---------------------------------------| 3.00% [3/100 00:00<00:17] |X---------------------------------------| 4.00% [4/100 00:00<00:12] |XX--------------------------------------| 5.00% [5/100 00:00<00:10] |XX--------------------------------------| 6.00% [6/100 00:00<00:08] |XXX-------------------------------------| 8.00% [8/100 00:00<00:06] |XXXX------------------------------------| 10.00% [10/100 00:00<00:04] |XXXXX-----------------------------------| 13.00% [13/100 00:00<00:03] |XXXXXX----------------------------------| 17.00% [17/100 00:00<00:02] |XXXXXXXXX-------------------------------| 23.00% [23/100 00:00<00:01] |XXXXXXXXXXXX----------------------------| 31.00% [31/100 00:00<00:01] |XXXXXXXXXXXXXXXX------------------------| 42.00% [42/100 00:00<00:00] |XXXXXXXXXXXXXXXXXXXXXX--

┌ Warning: Benchmark failed for julia
│   exception = CompositeException(Any[TaskFailedException(Task (failed) @0x0000018782cc4010)])
│   stacktrace = Union{Ptr{Nothing}, Base.InterpreterIP}[Ptr{Nothing} @0x0000018727efac1b, Ptr{Nothing} @0x0000018711b8a74a, Ptr{Nothing} @0x0000018711b9ae5a, Ptr{Nothing} @0x0000018711c30862, Ptr{Nothing} @0x0000018711c30a26, Ptr{Nothing} @0x0000018711c30d3c, Ptr{Nothing} @0x0000018727ed7e7f, Ptr{Nothing} @0x0000018727ed8159, Ptr{Nothing} @0x0000018727ed818f, Ptr{Nothing} @0x00007ffd621f708a, Ptr{Nothing} @0x0000018727ec4f5d, Ptr{Nothing} @0x0000018727ed0521, Ptr{Nothing} @0x0000018727ed1611, Ptr{Nothing} @0x0000018727ed1c85, Ptr{Nothing} @0x0000018727ed1cc4, Ptr{Nothing} @0x00007ffd62206c3a, Ptr{Nothing} @0x00007ffd6220675c, Ptr{Nothing} @0x00007ffd622072f8, Ptr{Nothing} @0x00007ffd622080fe, Base.InterpreterIP in top-level CodeInfo for Main at statement 0, Ptr{Nothing} @0x00007ffd62223ccb, Ptr{Nothing} @0x00007ffd6222463e, Ptr{Nothing} @0x00007ffd62225

 |----------------------------------------| 0.00% [0/100 00:00<?] |----------------------------------------| 1.00% [1/100 00:00<00:58] |----------------------------------------| 2.00% [2/100 00:00<00:29] |X---------------------------------------| 3.00% [3/100 00:00<00:19] |X---------------------------------------| 4.00% [4/100 00:00<00:14] |XX--------------------------------------| 5.00% [5/100 00:00<00:11] |XX--------------------------------------| 6.00% [6/100 00:00<00:09] |XXX-------------------------------------| 8.00% [8/100 00:00<00:06] |XXXX------------------------------------| 10.00% [10/100 00:00<00:05] |XXXXX-----------------------------------| 13.00% [13/100 00:00<00:03] |XXXXXX----------------------------------| 17.00% [17/100 00:00<00:02] |XXXXXXXX--------------------------------| 22.00% [22/100 00:00<00:02] |XXXXXXXXXXX-----------------------------| 29.00% [29/100 00:00<00:01] |XXXXXXXXXXXXXXX-------------------------| 38.00% [38/100 00:00<00:01] |XXXXXXXXXXXXXXXXXXX-----

┌ Warning: Benchmark failed for pykalman
│   exception = PyException(<py ValueError('array must not contain infs or NaNs')>)
│   stacktrace = Union{Ptr{Nothing}, Base.InterpreterIP}[Ptr{Nothing} @0x00007ffd58cc42e7, Ptr{Nothing} @0x0000018711c1e8ae, Ptr{Nothing} @0x0000018711c31510, Ptr{Nothing} @0x0000018711c3162f, Ptr{Nothing} @0x0000018711c31924, Ptr{Nothing} @0x0000018727ed7e7f, Ptr{Nothing} @0x0000018727ed8159, Ptr{Nothing} @0x0000018727ed818f, Ptr{Nothing} @0x00007ffd621f708a, Ptr{Nothing} @0x0000018727ec4f5d, Ptr{Nothing} @0x0000018727ecd9ad, Ptr{Nothing} @0x0000018727ed171c, Ptr{Nothing} @0x0000018727ed1c85, Ptr{Nothing} @0x0000018727ed1cc4, Ptr{Nothing} @0x00007ffd62206c3a, Ptr{Nothing} @0x00007ffd6220675c, Ptr{Nothing} @0x00007ffd622072f8, Ptr{Nothing} @0x00007ffd622080fe, Base.InterpreterIP in top-level CodeInfo for Main at statement 0, Ptr{Nothing} @0x00007ffd62223ccb, Ptr{Nothing} @0x00007ffd6222463e, Ptr{Nothing} @0x00007ffd6222565f, Ptr{Nothing} @0x0000018727e566e0, Ptr{